### Configure Ollama with structured outputs ###

In [4]:
import os
import pandas as pd
import numpy as np
import logging
import time
from pathlib import Path

logger = logging.getLogger(__name__)

import ollama

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import process_prompt
from llmt.openai import OpenAIModel, OpenAI
from llmt.openai import MentalHealth, OutpatientServices, InpatientServices, create_messages
from llmt.performance import Performance

print(llmt.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


AttributeError: module 'llmt' has no attribute '__version__'

In [8]:
from ollama import Client
client = Client(
  host='http://ollama:11434',
  headers={'x-some-header': 'some-value'}
)

In [37]:
model = 'llama2:7b'
client.pull(model)
ollama_models = client.list().get('models')
ollama_model_list = [model['model'] for model in ollama_models]
print(ollama_model_list)

['llama2:7b']


In [39]:
response = client.chat(model=model, messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response)

model='llama2:7b' created_at='2025-05-04T22:53:52.087127538Z' done=True done_reason='stop' total_duration=6026369351 load_duration=3555888 prompt_eval_count=26 prompt_eval_duration=27564515 eval_count=221 eval_duration=5994644443 message=Message(role='assistant', content="\nThe sky appears blue because of a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths. This is known as Rayleigh scattering.\n\nAs a result of this scattering, the blue light is dispersed throughout the atmosphere, giving the sky its characteristic blue color. The blue light scatters in all directions, but it is most visible when the sun is at a low angle in the sky, such as during sunrise or sunset.\n\nOther factors can also affect the color of the sky, such as air pollution, dust, and water 